#### Algorithmics for Data Mining Project 1
# Support Vector Machine
#### Ondrej Herman, Darryl Abraham

#### Importing Libraries

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
import os
import csv
from sklearn.preprocessing import LabelEncoder
import time

#### Importing the dataset

In [2]:
df = pd.read_csv('data/preprocessed_data.csv')
df.head()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode,album_name,artists,explicit,track_id,time_signature,track_name,key,track_genre
0,0.975633,-0.730859,-0.845908,-1.889980,-1.784744,-0.078993,1.831732,-0.504094,-0.591211,-0.798690,-1.489717,1,Ghost (Acoustic),Ben Woodward,False,4qPNDBW1i3p13qLCt0Ki3A,4,Ghost - Acoustic,1,acoustic
1,1.065299,-0.160332,-0.742186,-1.122669,-0.293288,-0.273826,-0.315499,-0.504112,-0.507167,-1.365688,-1.528312,1,To Begin Again,Ingrid Michaelson;ZAYN,False,1iJBSr7s7jYXzM8EGcbK5b,4,To Begin Again,0,acoustic
2,1.692961,-0.243214,-1.733304,-2.312994,-2.039252,-0.457309,1.774593,-0.503883,-0.428376,-1.276974,1.987859,1,Crazy Rich Asians (Original Motion Picture Sou...,Kina Grannis,False,6lfxq3CG4xtTiEg7opyCyx,3,Can't Help Falling In Love,0,acoustic
3,2.186125,-0.271919,0.295030,-0.788711,-0.282750,-0.303145,0.463399,-0.504112,-0.686285,-1.184403,-0.073348,1,Hold On,Chord Overstreet,False,5vjLSffimiIP26QG5WcN2K,4,Hold On,2,acoustic
4,1.827460,0.012776,-0.920818,-1.965518,-0.111952,-0.464875,1.630242,-0.504103,-0.642162,-1.533474,0.638339,1,Is There Anybody Out There?,A Great Big World;Christina Aguilera,False,6Vc5wAMmXdKIAM7WUoEb7N,3,Say Something,2,acoustic


#### Splitting the dataset into the Training set and Test set

In [13]:
# Stratified train test split on track_genre
X = df.drop(columns=['track_genre', 
                     'track_id', 
                     'track_name', 
                     'artists', 
                     'album_name'], 
            axis=1)
X['explicit'] = X['explicit'].astype(int)
y = df['track_genre']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### Training the Random Forest model

Define model

In [14]:
model_params = {
    'C': 1.0,
    'kernel': 'rbf',
    'degree': 3,
    'gamma': 'scale',
    'tol': 1e-3,
    'random_state': None
}
clf = svm.SVC(**model_params)

Tune by cross-validation

In [16]:
cv = 2
start = time.time()
scores = cross_val_score(clf, X_train, y_train, cv=cv, n_jobs=-1)
end = time.time()

#######################  SAVE  RESULTS  #######################

row = model_params.copy()
row['cv'] = cv; row['cv_scores'] = scores; row['mean_accuracy'] = scores.mean(); row['time'] = end-start

csv_file = 'svm_evaluation.csv'
file_exists = os.path.isfile(csv_file)

with open(csv_file, mode='a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=list(row.keys()))
    if not file_exists:
        writer.writeheader()
    writer.writerow(row)

#######################  PRINT  RESULTS  #######################

print(f'Time: {end-start}')
print(f'Cross-Validation Scores: {scores}')
print(f'Mean Accuracy: {scores.mean()}')

Time: 380.2782919406891
Cross-Validation Scores: [0.23166013 0.23372023]
Mean Accuracy: 0.23269018189126722


Tuning results
*N.B. if field is null, default value is used (check docs)*

In [17]:
eval = pd.read_csv('svm_evaluation.csv')
eval.tail(5)

,C,kernel,degree,gamma,tol,random_state,cv,cv_scores,mean_accuracy,time
0,1.0,rbf,3,scale,0.001,NaN,2,[0.23166013 0.23372023],0.23269,380.278292


#### Testing the model

In [ ]:
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f'Test Accuracy: {test_score}')